<a href="https://colab.research.google.com/github/karishita/MT2025011_MT2025052_Classification/blob/main/Model_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [ ]:
original=pd.read_csv('/content/ObesityDataSet_cleaned.csv')
given=pd.read_csv('/content/cleaned_data_given.csv')

In [ ]:
df=pd.concat([original,given],axis=0).reset_index(drop=True)

In [ ]:
df.drop(columns=['id'],inplace=True)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17644 entries, 0 to 17643
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype   
---  ------                          --------------  -----   
 0   Gender                          17644 non-null  category
 1   Age                             17644 non-null  float64 
 2   Height                          17644 non-null  float64 
 3   Weight                          17644 non-null  float64 
 4   family_history_with_overweight  17644 non-null  category
 5   FAVC                            17644 non-null  category
 6   FCVC                            17644 non-null  category
 7   NCP                             17644 non-null  category
 8   CAEC                            17644 non-null  category
 9   SMOKE                           17644 non-null  category
 10  CH2O                            17644 non-null  category
 11  SCC                             17644 non-null  category
 12  FAF               

In [ ]:
exclude_cols = ['Age', 'Height', 'Weight', 'target']

for col in df.columns:
    if col not in exclude_cols:
        df[col] = df[col].astype('category')


In [ ]:
df.describe()

,Age,Height,Weight,target
count,17644.000000,17644.000000,17644.000000,17644.000000
mean,23.875686,1.700129,87.641752,3.229823
std,5.751195,0.088362,26.350050,2.049784
min,14.000000,1.450000,39.000000,0.000000
25%,20.000000,1.630730,66.000000,1.000000
50%,22.771612,1.700000,84.000000,3.000000
75%,26.000000,1.763215,111.275646,5.000000
max,61.000000,1.980000,173.000000,6.000000


In [ ]:
X=df.iloc[:,0:16]
y=df.iloc[:,-1]

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X[['Age', 'Weight', 'Height']] = scaler.fit_transform(X[['Age', 'Weight', 'Height']])


In [ ]:
y

,target
0,1
1,1
2,1
3,2
4,3
...,...
17639,0
17640,0
17641,0
17642,2


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 10.1 MB/s eta 0:00:00


In [ ]:
df.columns

Index(['Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight',
       'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE',
       'CALC', 'MTRANS', 'target'],
      dtype='object')

In [ ]:
#catboost on combined

from catboost import CatBoostClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
cat_features=['Gender', 'family_history_with_overweight',
       'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE',
       'CALC', 'MTRANS']
base_model = CatBoostClassifier(
    cat_features=cat_features,
    eval_metric='Accuracy',
    random_seed=42,
    verbose=1,          # silence CatBoost during RandomizedSearchCV
    use_best_model=False,
    task_type='GPU'
)
param_dist = {
    'depth': [4, 5, 6, 7, 8, 9, 10],
    'learning_rate': [0.01, 0.03, 0.05, 0.1, 0.2],
    'iterations': [300],
    'l2_leaf_reg': [1, 3, 5, 7, 9],
    'border_count': [32, 50, 100, 200],  # number of splits for continuous features
    'bagging_temperature': [0, 0.2, 0.5, 1.0],
    'random_strength': [0.5, 1.0, 2.0]
}
cat_random = RandomizedSearchCV(
    estimator=base_model,
    param_distributions=param_dist,
    n_iter=30,             # number of parameter combinations to try
    scoring='accuracy',    # evaluation metric
    cv=3,                  # 5-fold cross-validation
    verbose=2,
    random_state=42,
    n_jobs=-1
)

cat_random.fit(X_train, y_train)

# 7️⃣ Print results
print("Best Parameters:", cat_random.best_params_)
print("Best Cross-Validation Accuracy:", cat_random.best_score_)

# 8️⃣ Evaluate best model on test data
best_cat = cat_random.best_estimator_
y_pred = best_cat.predict(X_test)

print("\n🎯 Test Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Fitting 3 folds for each of 30 candidates, totalling 90 fits
0:	learn: 0.7146298	total: 57.9ms	remaining: 17.3s
1:	learn: 0.7635848	total: 102ms	remaining: 15.1s
2:	learn: 0.7744952	total: 121ms	remaining: 12s
3:	learn: 0.7910733	total: 155ms	remaining: 11.5s
4:	learn: 0.8064470	total: 188ms	remaining: 11.1s
5:	learn: 0.8138151	total: 204ms	remaining: 10s
6:	learn: 0.8182784	total: 220ms	remaining: 9.19s
7:	learn: 0.8424371	total: 232ms	remaining: 8.45s
8:	learn: 0.8497343	total: 245ms	remaining: 7.92s
9:	learn: 0.8508679	total: 261ms	remaining: 7.56s
10:	learn: 0.8520014	total: 280ms	remaining: 7.35s
11:	learn: 0.8540560	total: 298ms	remaining: 7.16s
12:	learn: 0.8558980	total: 319ms	remaining: 7.04s
13:	learn: 0.8583068	total: 349ms	remaining: 7.13s
14:	learn: 0.8643996	total: 364ms	remaining: 6.92s
15:	learn: 0.8677294	total: 381ms	remaining: 6.75s
16:	learn: 0.8707758	total: 433ms	remaining: 7.21s
17:	learn: 0.8710592	total: 465ms	remaining: 7.28s
18:	learn: 0.8713425	total: 484ms	

In [ ]:
#Random Forest on combined
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
rf=RandomForestClassifier(random_state=42)
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [None, 5, 10, 15, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None],
    'bootstrap': [True, False]
}
rf_random = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=60,               # number of random combinations
    cv=5,                    # 5-fold cross-validation
    verbose=2,               # progress output
    random_state=42,
    n_jobs=-1,               # use all cores
    scoring='accuracy'       # can change to 'f1_macro' if classes imbalanced
)

rf_random.fit(X_train, y_train)
best_rf = rf_random.best_estimator_
y_pred = best_rf.predict(X_test)
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:", confusion_matrix(y_test, y_pred))

Fitting 5 folds for each of 60 candidates, totalling 300 fits

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.92      0.93       457
           1       0.86      0.89      0.87       535
           2       0.78      0.75      0.77       402
           3       0.80      0.83      0.82       437
           4       0.93      0.89      0.91       512
           5       0.96      0.98      0.97       534
           6       1.00      1.00      1.00       652

    accuracy                           0.90      3529
   macro avg       0.89      0.89      0.89      3529
weighted avg       0.90      0.90      0.90      3529

Test Accuracy: 0.9033720600736752
Confusion Matrix: [[419  36   2   0   0   0   0]
 [ 25 474  29   6   1   0   0]
 [  3  34 301  59   5   0   0]
 [  0  10  37 363  23   4   0]
 [  1   0  14  23 457  15   2]
 [  0   0   0   2   8 524   0]
 [  0   0   1   0   0   1 650]]


In [ ]:
#Knn on combined

# KNN on combined
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Initialize KNN
knn = KNeighborsClassifier()

# Define hyperparameter search space
param_dist = {
    'n_neighbors': list(range(1, 31)),           # number of neighbors
    'weights': ['uniform', 'distance'],          # weighting method
    'metric': ['euclidean', 'manhattan', 'minkowski'],  # distance metrics
    'p': [1, 2],                                 # power parameter for Minkowski
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']  # algorithm to compute neighbors
}

# Randomized search setup
knn_random = RandomizedSearchCV(
    estimator=knn,
    param_distributions=param_dist,
    n_iter=60,             # number of random combinations
    cv=5,                  # 5-fold cross-validation
    verbose=2,             # show progress
    random_state=42,
    n_jobs=-1,             # use all cores
    scoring='accuracy'     # use 'f1_macro' for imbalanced data
)

# Fit the model
knn_random.fit(X_train, y_train)

# Best model
best_knn = knn_random.best_estimator_

# Predictions
y_pred = best_knn.predict(X_test)

# Evaluation
print("\nBest Parameters:", knn_random.best_params_)
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Test Accuracy:", accuracy_score(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Fitting 5 folds for each of 60 candidates, totalling 300 fits

Best Parameters: {'weights': 'distance', 'p': 2, 'n_neighbors': 14, 'metric': 'manhattan', 'algorithm': 'ball_tree'}

Classification Report:
               precision    recall  f1-score   support

           0       0.85      0.88      0.86       457
           1       0.74      0.68      0.71       535
           2       0.65      0.59      0.62       402
           3       0.70      0.71      0.71       437
           4       0.80      0.81      0.80       512
           5       0.90      0.97      0.93       534
           6       0.99      1.00      0.99       652

    accuracy                           0.82      3529
   macro avg       0.80      0.81      0.80      3529
weighted avg       0.82      0.82      0.82      3529

Test Accuracy: 0.8203457069991499
Confusion Matrix:
 [[402  49   5   0   1   0   0]
 [ 60 365  68  28  11   3   0]
 [  9  56 237  61  36   3   0]
 [  2  22  39 311  47  16   0]
 [  2   3  13  39 415

In [ ]:
#XGBOOST on combined
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# ✅ XGBoost base model with categorical support
xgb = XGBClassifier(
    random_state=42,
    objective='multi:softmax',
    num_class=len(y.unique()),
    eval_metric='mlogloss',
    use_label_encoder=False,
    tree_method='hist',          # required for categorical data
    enable_categorical=True      # allow category dtype columns
)

#  parameter grid
param_dist = {
    'n_estimators': [200, 300, 400, 500, 600],
    'max_depth': [3, 5, 7, 9, 11],
    'learning_rate': [0.01, 0.03, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3],
    'min_child_weight': [1, 3, 5],
    'reg_alpha': [0, 0.01, 0.1, 0.5, 1.0],   # L1 regularization
    'reg_lambda': [0.5, 1, 1.5, 2, 3]        # L2 regularization
}

# Randomized Search setup
xgb_random = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=60,              # number of random combinations
    scoring='accuracy',     # 'f1_macro' if imbalanced
    cv=5,                   # 5-fold CV
    verbose=2,
    random_state=42,
    n_jobs=-1               # use all cores
)

# ✅ Train model
xgb_random.fit(X_train, y_train)

# ✅ Print best hyperparameters
print("Best Parameters:", xgb_random.best_params_)
print("Best Cross-Validation Accuracy:", xgb_random.best_score_)

# ✅ Evaluate on test set
best_xgb = xgb_random.best_estimator_
y_pred = best_xgb.predict(X_test)

print("\n Test Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))



Fitting 5 folds for each of 60 candidates, totalling 300 fits


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [16:48:53] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Best Parameters: {'subsample': 0.8, 'reg_lambda': 2, 'reg_alpha': 0.01, 'n_estimators': 300, 'min_child_weight': 3, 'max_depth': 9, 'learning_rate': 0.03, 'gamma': 0.2, 'colsample_bytree': 0.6}
Best Cross-Validation Accuracy: 0.9119376549769749

 Test Accuracy: 0.9104562198923207

Confusion Matrix:
 [[422  33   1   0   1   0   0]
 [ 24 476  27   8   0   0   0]
 [  3  28 316  46   9   0   0]
 [  0   9  32 369  23   4   0]
 [  1   0  12  25 456  16   2]
 [  0   0   0   1   9 524   0]
 [  0   0   0   0   2   0 650]]

Classification Report:
               precision    recall  f1-score   support

           0       0.94      0.92      0.93       457
           1       0.87      0.89      0.88       535
           2       0.81      0.79      0.80       402
           3       0.82      0.84      0.83       437
           4       0.91      0.89      0.90       512
           5       0.96      0.98      0.97       534
           6       1.00      1.00      1.00       652

    accuracy          

In [ ]:
df1=pd.read_csv('/content/test_cleaned.csv')
df1[['Age', 'Weight', 'Height']] = scaler.transform(df1[['Age', 'Weight', 'Height']])
exclude_cols = ['Age', 'Height', 'Weight']

for col in df1.columns:
    if col not in exclude_cols:
        df1[col] = df1[col].astype('category')

In [ ]:
def reverse_target_mapping(value):
    if value == 1:
        return 'Normal_Weight'
    elif value == 2:
        return 'Overweight_Level_I'
    elif value == 3:
        return 'Overweight_Level_II'
    elif value == 4:
        return 'Obesity_Type_I'
    elif value == 0:
        return 'Insufficient_Weight'
    elif value == 5:
        return 'Obesity_Type_II'
    elif value == 6:
        return 'Obesity_Type_III'


In [ ]:
y_pred1=best_cat.predict(df1.iloc[:,1:])
y_pred_labels = pd.Series(y_pred1.flatten()).apply(reverse_target_mapping)
df1['WeightCategory']=y_pred_labels
df_new=df1[['id','WeightCategory']]
df_new.to_csv('submission_final.csv',index=False)